# Notebook 1: Filling histograms with processHandler

In this notebook there is a simple example of how Galapago fills a histogram with an instance of the class ```processHandler```. This class manages:
1. Which histograms are declared in in first place
2. Which histograms are filled and how
3. The selection that is applied to every object to fill the histogram
4. Major selections to define the different regions of the analysis

This 4 things will be defined in several functions of the ```processHandler``` class, so to make variations or tune the filling processes this class must be modified accordingly.

<em>Note: This is a little tricky, since the class must be altered for every configuration. However it also gives the chance to be more flexible when defining the histograms and how they are filled.</em>

In [1]:
import os

if 'notebooks' in os.getcwd():
    os.chdir('..')

## 1.0 - processHandler structure

A very simple version of the ```processHandler``` class is defined to provide an overall view of its functioning. We first import everything that is needed:

In [2]:
import math
import os
import ROOT as r
from ROOT import TVector3, TLorentzVector
import include.CutManager as CutManager
import include.Sample as Sample
import include.helper as helper
import numpy as np

Welcome to JupyROOT 6.20/06


And a very simple structured ```processHandler``` class is declared:

1. Each process is <strong>initialised</strong> with:
    * The output directory where the ```.root``` files will be stored
    * Four identifiers for the ```Tree``` name, the ```Block``` name, the ```Sample``` name and the sample number ('0' if only one)
    * The ```lumiweight``` to weight the events 
    * Bool parameter to label the process as data or Monte Carlo
   
   Each process will fill a set of predefined histograms, that are declared when the process is first created.
    
    
2. <strong>The set of histograms is declared</strong> in the ```declareDimuonHistograms()``` and ```declareDielectronsHistograms()``` functions.


3. This set of histograms will be created and filled for <strong>every region especified</strong> in ```declareDimuonRegions()``` and ```declareDielectronRegions()```.
    * Each region is specified with a name and a cut (both strings) that is evaluated when filling the histograms
    
    
4. The core of the class is the ```processEvent()``` function, that is called on the ```processHandler``` instance, giving a certain ```ROOT.TTree.Event``` as a parameter (refered as ev). This event is used to <strong>fill one entry of the histograms</strong>.


5. The functions ```processDimuons()``` and ```processDielectrons()``` are called inside ```processEvent()``` for every event. They <strong>search the index of the MM/EE candidate</strong> that will be chosen to fill the histograms (loops over all the candidates applying cuts).


6. The functions ```fillDimuonHistograms()``` and ```fillDielectronHistograms()``` receive the previous found indexes as parameters. Inside these functions, <strong>the histograms are filled</strong> by using these indexes.


7. The ```Write()``` function is called when the filling process has finished e.g. the loop over the tree entries has ended, and <strong>save the already filled histograms</strong> in a ```.root``` file.


In [3]:
class processHandler:
    
    def __init__(self, outdir, treename, blockname, samplename, samplenumber, lumiweight, isdata):

        ### Inputs
        ###
        ###   outdir: Path where the root file will be stored
        ###   treename: Name of the (Galapago) Tree (MC; DATA; SI)
        ###   blockname: Name of the (Galapago) Block
        ###   samplename: Name of the sample
        ###   samplenumber: Number of the sample file/ttree
 

        #### ----------------------------------
        #### ---- Class parameter declaration
        #### ----------------------------------
         
        self.outdir = outdir
        if self.outdir[-1] != '/': self.outdir = self.outdir + '/'
        self.filename = self.outdir + '{0}__{1}__{2}__{3}.root'.format(treename, blockname, samplename, str(samplenumber))
        self.treename = treename
        self.blockname = blockname
        self.samplename = samplename
        self.samplenumber = samplenumber
        self.lumiweight = lumiweight
        self.isdata = isdata
        self.cm = CutManager.CutManager()
        self.sufix = '__{0}__{1}__{2}__{3}'.format(treename, blockname, samplename, str(samplenumber))
            

        #### ------------------------------
        #### ---- Regions initialization
        #### ------------------------------
    
        self.dielectronRegions = [] # region name : region cut
        self.declareDielectronRegions()
        
        self.dimuonRegions = [] # region name : region cut
        self.declareDimuonRegions()
    
    
        #### -------------------------------
        #### ---- Histogram initialization
        #### -------------------------------
        #### One set of histograms in each region defined
        
        for region in self.dielectronRegions:
            region_name = region[0]
            self.declareDielectronHistograms(region_name)
            
        for region in self.dimuonRegions:
            region_name = region[0]
            self.declareDimuonHistograms(region_name)
    
    
        #### --------------------------------
        #### ---- Apply sumw2 to histograms
        #### --------------------------------

        for attr, value in self.__dict__.items():
            if attr[0] == 'h': value.Sumw2()
                
                
                
    #### ------------------------
    #### --
    #### ---- declareHistograms
    #### --
    #### ------------------------

    def declareDimuonHistograms(self, region):

        exec("self.hMM{0}_nBSMM = r.TH1F('hMM{0}_nBSMM' + self.sufix, ';Number of MM candidates;', 4, 0, 4)".format(region))
        exec("self.hMM{0}_trackIxy = r.TH1F('hMM{0}_trackIxy' + self.sufix, ';Dimuon |d_{{0}}|/#sigma_{{d}};', 40, 0, 40)".format(region))
        
    def declareDielectronHistograms(self, region):
        
        exec("self.hEE{0}_nBSEE = r.TH1F('hEE{0}_nBSEE' + self.sufix, ';Number of EE candidates;', 4, 0, 4)".format(region))
        exec("self.hEE{0}_trackIxy = r.TH1F('hEE{0}_trackIxy' + self.sufix, ';Dielectron |d_{{0}}|/#sigma_{{d}};', 40, 0, 40)".format(region))
        
        
        
    #### ------------------------
    #### --
    #### ---- declareRegions
    #### --
    #### ------------------------  
    
    def declareDimuonRegions(self):
        
        MM_OSSRcut = self.cm.AddList([self.cm.MM_OS, self.cm.MM_iso2l, self.cm.MM_dPhiforward])

        self.dimuonRegions.append(['SROS', MM_OSSRcut])

    def declareDielectronRegions(self):
        
        EE_OSSRcut = self.cm.AddList([self.cm.EE_OS, self.cm.EE_iso2l, self.cm.EE_dPhiforward])
        
        self.dielectronRegions.append(['SROS', EE_OSSRcut])
        
        
    #### ----------------------------
    #### --
    #### ---- Core event processing
    #### --
    #### ----------------------------

    def processEvent(self, ev):


        # Weight definition:
        if not self.isdata:
            weight = self.lumiweight*ev.wPU*ev.genWeight/abs(ev.genWeight)
            weightnoPU = self.lumiweight*ev.genWeight/abs(ev.genWeight)
        else:
            weight = 1


        # Test PV acceptance:
        if not ev.PV_passAcceptance: return


        # Dimuon processing

        try:

            mm_maxIxy = -99
            mm_maxIxy, nBSMM = self.processDimuons(ev)

            if not mm_maxIxy < 0:

                imm = mm_maxIxy
                for region in self.dimuonRegions:
                    if eval(region[1]):
                        self.fillDimuons(ev, weight, region[0], mm_maxIxy, nBSMM)

        except AttributeError:
            print('There is some collections missed in this file: Dimuon histograms will be empty')

            
        # Dielectron processing

        try:

            ee_maxIxy = -99
            ee_maxIxy, nBSEE = self.processDielectrons(ev)

            if not ee_maxIxy < 0:

                iee = ee_maxIxy
                for region in self.dielectronRegions:
                    if eval(region[1]):
                        self.fillDielectrons(ev, weight, region[0], ee_maxIxy, nBSEE)

        except AttributeError:
            print('There is some collections missed in this file: Dielectron histograms will be empty')
            
            
    
    #### ------------------------
    #### --
    #### ---- DiMuon Processing
    #### --
    #### ------------------------

    def processDimuons(self, ev):


        #### ----------------------------
        #### ---- Select a MM candidate
        #### ----------------------------

        ### -> Events just passing the trigger and having a valid DG pair
        if not ev.Flag_HLT_L2DoubleMu28_NoVertex_2Cha_Angle2p5_Mass10 or ev.nDMDM == 0: return -99, -99

        ### -> Find the MM pair with maximum Ixy that passes the cuts
        mm_maxIxy = -99
        maxIxy = -1
        nBSMM = 0

        for i in range(0, ev.nDMDM):

            imm = i
            if not eval(self.cm.MM_ID): continue
            if not eval(self.cm.MM_cosAlpha0p8): continue
            if not eval(self.cm.MM_mass15): continue
            if not eval(self.cm.MM_normChi2_10): continue
            # dR cut on muons is missing

            if eval(self.cm.MM_iso2l) and eval(self.cm.MM_OS): nBSMM+=1

            if ev.DMDM_trackIxy_PV[i] > maxIxy:
                maxIxy = ev.DMDM_trackIxy_PV[i]
                mm_maxIxy = i

        # return the index of the selected MM and the number of BS MM's:
        return mm_maxIxy, nBSMM
    
    
    #### ---------------------
    #### --
    #### ---- DiMuon filling
    #### --
    #### ---------------------

    def fillDimuons(self, ev, weight, region, mm_maxIxy, nBSMM):

        exec("self.hMM{0}_nBSMM.Fill(nBSMM, weight)".format(region))
        exec("self.hMM{0}_trackIxy.Fill(ev.DMDM_trackIxy_PV[mm_maxIxy], weight)".format(region))
        
        
    #### ----------------------------
    #### --
    #### ---- DiElectron Processing
    #### --
    #### ----------------------------

    def processDielectrons(self, ev):

        if ev.Flag_HLT_L2DoubleMu28_NoVertex_2Cha_Angle2p5_Mass10 or not ev.Flag_HLT_Photon42_R9Id85_OR_CaloId24b40e_Iso50T80L_Photon25_AND_HE10_R9Id65_Eta2_Mass15 or ev.nEE == 0: return -99, -99

        ### -> Find the EE pair with maximum Ixy that passes the cuts
        ee_maxIxy = -99
        maxIxy = -1
        nBSEE = 0
        for i in range(0, ev.nEE):

            iee = i
            if not eval(self.cm.EE_eta2): continue
            if not eval(self.cm.EE_leadingPt45): continue
            if not eval(self.cm.EE_subleadingPt28): continue
            if not eval(self.cm.EE_leadingEt45): continue
            if not eval(self.cm.EE_subleadingEt28): continue
            if not eval(self.cm.EE_normChi2_10): continue
            if not eval(self.cm.EE_mass15): continue

            if eval(self.cm.EE_iso2l) and eval(self.cm.EE_OS): nBSEE+=1

            if ev.EE_trackIxy_PV[i] > maxIxy:
                maxIxy = ev.EE_trackIxy_PV[i]
                ee_maxIxy = i

        # return the index of the selected MM and the number of BS MM's:
        return ee_maxIxy, nBSEE


    #### -------------------------
    #### --
    #### ---- DiElectron filling
    #### --
    #### -------------------------

    def fillDielectrons(self, ev, weight, region, ee_maxIxy, nBSEE):

        exec("self.hEE{0}_nBSEE.Fill(nBSEE, weight)".format(region))
        exec("self.hEE{0}_trackIxy.Fill(ev.EE_trackIxy_PV[ee_maxIxy], weight)".format(region))

        
        
    #### ------------------------------
    #### --
    #### ---- End of process: writing
    #### --
    #### ------------------------------
        
    def Write(self):

        while True:
            d = os.path.dirname(self.filename)
            try:
                if not os.path.exists(d): os.makedirs(d)
                break
            except OSError as e:
                if e.errno != os.errno.EEXIST:
                    raise
                print("Sleeping...")
                time.sleep(1.0)
                pass

        output = r.TFile(self.filename, 'RECREATE')

        for attr, value in self.__dict__.items():
            if attr[0] == 'h': value.Write()


        output.Close()


            

## 1.1 - Declaring an instance for a sample

Once the ```processHandler``` class is define, we get a tree for filling a set of two histograms of candidates in the OS forward region:
    * The number of baseline candidates nBSEE/nBSMM
    * The transverse impact parameter significance of the chosen EE/MM candidate trackIxy
    
We read the ```HXX_1000_150_100``` sample by creating a ```Tree```. This sample is composed of only one ```.root``` file.

From the declared ```Tree``` can get the necessary info for creating the process:

In [4]:
# Access the Tree of one sample and print its information

treeSI = Sample.Tree( fileName = helper.selectSamples('dat/Samples_cern_fillingv2.dat', ['HXX_1000_150_100'], 'SI'), name = 'SI', isdata = 0 )
treeSI.printTree()


selectSamples for  SI : List Of Samples: ['HXX_1000_150_100']
---> Found a match for HXX_1000_150_100 : HXX_1000_150_100mm   HXX(1000,150,100) /eos/user/f/fernance/LLP_Analysis/NTuples/2016_v3/HXX_1000_150_100mm/ 0.7739 , matchesName_= False , matchesRegExp True
######
Tree Name:  SI
Tree IsData:  0
######
This Tree contains the following Blocks
####################
Block Name:  HXX_1000_150_100mm
Block Color:  632
Block IsData:  0
####################
This block contains the following Samples
#################################
Sample Name:  HXX_1000_150_100mm
Sample Location:  /eos/user/f/fernance/LLP_Analysis/NTuples/2016_v3/HXX_1000_150_100mm/
Sample XSection:  0.7739
Sample IsData:  0
Sample LumWeight:  4.060335781741868e-06
#################################


And also we can get the ```ROOT.TTree``` itself:

In [5]:
# Get the TTree

tree_block = treeSI.blocks[0]
tree_sample = tree_block.samples[0]
ttree = tree_sample.ttrees[0]

print("Tree entries: ", ttree.GetEntries())

Tree entries:  190600


The process is created:

In [6]:
# Declare the processHandler instance with the sample information (sample, block, lumiweight for normalization... etc)

process = processHandler(outdir = 'output_histograms2/', 
                         treename = 'SI', 
                         blockname = 'HXX_1000_150_100mm', 
                         samplename = 'HXX_1000_150_100mm', 
                         samplenumber = '0', 
                         lumiweight = 4.060335781741868e-06, 
                         isdata = False)

We run over all the events of the ```ROOT.TTree``` and the use the ```processEvent()``` function to fill <strong>one entry</strong> of the histograms. The ```ROOT.Tree.Event``` is given as a parameter.

<em>(Notice that the functions ```fillDimuonHistograms()``` and ```fillDielectronHistograms()``` are called inside ```processEvent()```)

In [7]:
for n,ev in enumerate(ttree):

    process.processEvent(ev)

Once finished, we write the filled histograms of the process in a file:

In [8]:
process.Write()

And we check that this file is created:

In [9]:
!ls -l output_histograms2/

total 7
-rw-r--r--. 1 fernance 1399 6781 Dec 26 15:30 SI__HXX_1000_150_100mm__HXX_1000_150_100mm__0.root
